In [1]:
import os, config, pickle, nbimporter, _1_extract, numpy as np, pandas as pd
Species, Data = _1_extract.Species, _1_extract.Data

Importing Jupyter notebook from _1_extract.ipynb


# Processing the lxmx data

## Goals:
* construct projection matrix $P$ for data
* for each $P$ compute dominant & subdominant eigenvalues and their logs, $r, r_1$
* for each $P$ compute $T_c, T_g, V, r_{0a}, r_{0b}, d, G, K$ (see theory for reference)

In [2]:
SPECIES = config.load_pickle(os.path.join(config.OUTPUT_DIR, 'species.pkl'))

## Auxillary Classes

#### Processing
This class will handle all of the goals mentioned above and augment the Data object created in the data extraction step.

In [55]:
class Processing:
    
    @staticmethod
    def process(species):
        for data in species.data:
            matrix = data.matrix
            ages = data.ages
            l_a = matrix['l(a)'].to_numpy()
            f_a = matrix['f(a)'].to_numpy()
            p_a = matrix['p(a)'].to_numpy()
            data.leslie = Processing.generate_leslie(p_a, f_a)
            eigs = Processing.compute_eig(data.leslie)
            data.eigen = {'vals': eigs[0], 'right': eigs[1], 'left': eigs[2], 'r_i': eigs[3]}
            data.eigen['damping'] = np.exp(data.eigen['r_i'][1] - data.eigen['r_i'][0])
            data.derivatives = Processing.compute_derivatives(l_a, f_a, ages)
            
    @staticmethod
    def generate_leslie(p_a, f_a):
        N = p_a.shape[0]
        leslie = np.zeros((N, N))
        leslie[0] = f_a
        np.fill_diagonal(leslie[1:, :-1], p_a)
        return leslie

    @staticmethod
    def compute_eig(leslie):
        #RIGHT
        r_vals, r_vecs = np.linalg.eig(leslie)
        ix_r = r_vals.argsort()
        ix_r = ix_r[::-1]
        r_vals = r_vals[ix_r]
        r_vecs = r_vecs[ix_r]
        #LEFT
        l_vals, l_vecs = np.linalg.eig(leslie.T)
        ix_l = l_vals.argsort()
        ix_l = ix_l[::-1]
        l_vals = l_vals[ix_l]
        l_vecs = l_vecs[ix_l]
        assert np.linalg.norm(l_vals - r_vals) < .00001 #check eigenvalues are the same
        r_eigs = r_vecs[:2]
        l_eigs = l_vecs[:2]
        vals = r_vals[:2]
        r_i = np.log(np.real(vals))
        return vals, r_eigs, l_eigs, r_i 
    
    @staticmethod
    def compute_derivatives(l_a, f_a, ages):
        derivatives = {}
        derivatives['R_0'] = R_0 = np.sum(l_a * f_a)
        derivatives['T_c'] = T_c =  1 / R_0 * np.sum(ages * l_a * f_a)
        derivatives['V'] = V = 1 / R_0 * np.sum((ages - T_c)**2 * l_a * f_a)
        derivatives['G'] = G = 1 / (R_0 * V**1.5) * np.sum((ages - T_c)**3 * l_a * f_a)
        derivatives['K'] = K =  1 / (R_0 * V**2) * np.sum((ages - T_c)**4 * l_a * f_a)
        derivatives['r_0a'] = r_0a = np.log(R_0)/T_c #without dispersion
        derivatives['r_0b'] = r_0a + V*np.log(R_0)**2/(T_c**3)  #with dispersion
        return derivatives
    
    @staticmethod
    def check_derivatives(species):
        for data in species.data:
            T_c_reported = data.reported['T_c'][1]
            R_0_reported = data.reported['R_0']
            V_reported = data.reported['V']
            exceptions = []
            if not (abs(data.derivatives['T_c'] - T_c_reported) < .01):
                exceptions.append('T_c calculated: {}, reported: {}'.format(data.derivatives['T_c'], T_c_reported))
            if not (R_0_reported is None or abs(data.derivatives['R_0'] - R_0_reported) < .01): 
                exceptions.append('R_0 calculated: {}, reported: {}'.format(data.derivatives['R_0'], R_0_reported))
            if not (V_reported is None or abs(data.derivatives['V'] - V_reported) < .01): 
                exceptions.append('V calculated: {}, reported: {}'.format(data.derivatives['V'], V_reported))
            if len(exceptions) > 0:
                combined = '; '
                combined = combined.join(exceptions)
                raise Exception('For {}, {}'.format(species, combined))
                

In [56]:
for s in SPECIES:
    if s == 'Spermophilus lateralis':
        continue
    Processing.process(SPECIES[s])
    try:
        Processing.check_derivatives(SPECIES[s])
    except Exception as e:
        print('**********')
        print(e)


**********
For Lepus europaeus: 1 data entries, notes: ['age in months'], T_c calculated: 13.279863097655307, reported: 1.217; V calculated: 13.720307623123851, reported: 0.264
**********
For Capreolus capreolus: 2 data entries, notes: [], T_c calculated: 4.555876774257954, reported: 4.754; V calculated: 3.9787695061941166, reported: 4.814
**********
For Sylvilagus floridanus: 1 data entries, notes: ['age in months', 'for 1 litter/year but several are produced (3.5)'], T_c calculated: 9.183254625265562, reported: 0.763; V calculated: 36.19810073077049, reported: 0.253
**********
For Pteropus conspicillatus: 1 data entries, notes: [], T_c calculated: 4.001943479485495, reported: 4.76; V calculated: 4.921872103086531, reported: 3.302
**********
For Lynx rufus: 1 data entries, notes: [], T_c calculated: 2.169733372399648, reported: 2.67
**********
For Arctocephalus australis: 1 data entries, notes: [], T_c calculated: 6.323251800587961, reported: 5.823
**********
For Liomys adspersus: 1 d

/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in log
/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


## Working with the Species objects

You can acccess the calculated derivatives as follows:

In [60]:
s = 'Capreolus capreolus'

In [62]:
SPECIES[s]

Capreolus capreolus: 2 data entries, notes: []

In [63]:
SPECIES[s].data[1].derivatives

{'R_0': 1.3652699999999998,
 'T_c': 4.190412885363336,
 'V': 3.8837257936206915,
 'G': 1.1266838507251522,
 'K': 4.465624285201734,
 'r_0a': 0.07430108196783416,
 'r_0b': 0.0794176884752565}

In [64]:
SPECIES[s].data[1].reported

{'T_c': [None, 4.19], 'V': 3.884, 'R_0': None}

In [66]:
SPECIES[s].data[1].eigen['r_i']

array([ 0.06143377, -0.73217181])

In [68]:
SPECIES[s].data[1].eigen['damping']

0.4522113685415103

In [59]:
save = True
if save:
    config.save_pickle(SPECIES, os.path.join(config.OUTPUT_DIR, 'species_with_derivatives.pkl'))

## Bugs

In [8]:
SPECIES['Spermophilus lateralis'].data[2].matrix

,l(a),f(a),p(a)
age,,,
0.0,1.000,0.00,0.223
1.0,0.223,2.17,0.296
2.0,0.066,3.19,0.455
3.0,0.030,3.17,>1
4.0,0.036,2.93,0.694
5.0,0.025,3.40,0.2
6.0,0.005,3.50,0


In [6]:
Processing.process(SPECIES['Ursus arctos horribilis'])

/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in log
/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [25]:
Processing.check_derivatives(SPECIES['Lepus europaeus'])

AssertionError: For Lepus europaeus: 1 data entries, notes: age in months, T_c calculated: 13.279863097655307, reported: 1.217